# Coursera_Capstone


In [2]:
#import packages that may e needed
import pandas as pd
import numpy as np

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

import matplotlib.pyplot as plt
%matplotlib inline 

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



PART 1, DATA CLEANSING

In [92]:
#define a variable url to store htm link there 
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#import data from html and pass url as an argument. As read produces a list of a data frames we further need to access desired one via index [0] 
data = pd.read_html(url)[0]
#give names to columns
data.columns = ['Postcode','Borough','Neighbourhood']
data


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [93]:
#ignore 'not assigned'
data = data[data.Borough != 'Not assigned']
data

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [94]:
#condition on the repeating, use groupby to gtoup y postcode and use aggregatefunction to gather values together & reset index from to get a nice looking table with 3 columns
data = data.groupby("Postcode").agg(lambda x:','.join(set(x))).reset_index()
data

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [106]:
#Now,if cell has a borough but neighborhood is na, then the neighborhood == borough
#Loop through the data frame
for i, j in data.iterrows():
    if j['Neighbourhood']== 'Not Assigned' and j['Borough']!= 'Not Assigned':
        j['Neighbourhood']= j['Borough']
data       

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [107]:
data.shape

(103, 3)